# Cybershuttle Molecular Dynamics SDK

This SDK allows users to define, plan, and execute molecular dynamics experiments with ease. In this notebook, we demonstrate how to authenticate, set up a NAMD experiment, add replicas, create an execution plan, and monitor the execution. The experiment object `exp` is an instance of `cybershuttle.base.Experiment`.

In [1]:
import cybershuttle as cs
from cybershuttle import md

## Authenticate for Remote Execution

To authenticate for remote execution, use the `cs.auth.login()` method. This method will prompt you to enter your credentials and authenticate your session.

In [2]:
cs.auth.login()
token = cs.auth.context.access_token

User code: LHZI-XXPX
Please authenticate by visiting: https://auth.cybershuttle.org/realms/10000000/device?user_code=LHZI-XXPX
Waiting for authorization...
Authorization successful!


## Define a NAMD Experiment

To define a NAMD experiment, we initialize an instance of the `cybershuttle.base.Experiment` class with the required parameters such as the name of the experiment, and the input PDB and PSF files. The experiment object `exp` is created as shown below:

In [4]:
exp = md.NAMD.initialize(
    name="yasith_namd_experiment",
    config_file="data/pull.conf",
    pdb_file="data/structure.pdb",
    psf_file="data/structure.psf",
    other_files=[
      "data/b4pull.pdb",
      "data/b4pull.restart.coor",
      "data/b4pull.restart.vel",
      "data/b4pull.restart.xsc",
      "data/par_all36_water.prm",
      "data/par_all36_prot.prm",
    ]
)

## Add Replicas for NAMD Experiment

To add replicas to the NAMD experiment, we use the `add_replica()` method of the `exp` object. This method adds a replica to the experiment. In this example, we add 4 replicas as shown below:

In [ ]:
runtimes = cs.runtime.query()

# always 0th index is local runtime inside
# 0th runtime requires docker to be installed

display(runtimes)

[Mock(id='mock', args={})]

In [ ]:
# choose between any replicas user has access to
# exp.add_replica(*runtimes)

# OR choose any replica that is not local
exp.add_replica(*runtimes[1:])

# OR use only a specific replica
# exp.add_replica(runtimes[1])

## Create Execution Plan

To create an execution plan for the experiment, use the `plan()` method of the `exp` object. This method will generate a plan for the experiment, which can then be described using the `describe()` method. The code to create and describe the plan is shown below:

In [6]:
plan = exp.plan()  # this will create a plan for the experiment
plan.describe()  # this will describe the plan

Task(app_id=namd, inputs={'config_file': 'data/pull.conf', 'pdb_file': 'data/structure.pdb', 'psf_file': 'data/structure.psf', 'other_files': ['data/b4pull.pdb', 'data/b4pull.restart.coor', 'data/b4pull.restart.vel', 'data/b4pull.restart.xsc', 'data/par_all36_water.prm', 'data/par_all36_prot.prm']}, runtime=Remote(args={'cluster': 'expanse', 'partition': 'shared', 'profile': 'grprsp-1'}))


In [ ]:
# can also save the plan to a file, or load a plan from a file
plan.save_json("plan.json")
plan = cs.plan.Plan.load_json("plan.json")
plan.describe()

Task(app_id=namd, inputs={'config_file': 'data/pull.conf', 'pdb_file': 'data/structure.pdb', 'psf_file': 'data/structure.psf', 'other_files': ['data/b4pull.pdb', 'data/b4pull.restart.coor', 'data/b4pull.restart.vel', 'data/b4pull.restart.xsc', 'data/par_all36_water.prm', 'data/par_all36_prot.prm']}, runtime=Remote(args={'cluster': 'expanse', 'partition': 'shared', 'profile': 'grprsp-1'}))


## Execute the Plan

In [8]:
plan.run()

Preparing execution plan...
Task(app_id=namd, inputs={'config_file': 'data/pull.conf', 'pdb_file': 'data/structure.pdb', 'psf_file': 'data/structure.psf', 'other_files': ['data/b4pull.pdb', 'data/b4pull.restart.coor', 'data/b4pull.restart.vel', 'data/b4pull.restart.xsc', 'data/par_all36_water.prm', 'data/par_all36_prot.prm']}, runtime=Remote(args={'cluster': 'expanse', 'partition': 'shared', 'profile': 'grprsp-1'}))
Confirming execution plan...
Starting tasks...
{'project_name': 'Default Project', 'app_name': 'namd', 'experiment_name': 'NAMD_from_md_sdk', 'experiment_description': 'Testing MD-SDK for December Workshop', 'resource_host_name': 'login.expanse.sdsc.edu', 'group_resource_profile_name': 'Default', 'storage_resource_name': 'iguide-cybershuttle.che070035.projects.jetstream-cloud.org', 'node_count': 1, 'total_cpu_count': 16, 'wall_time_limit': 15, 'queue_name': 'shared', 'gateway_id': 'default'}
{'configuration_file_location': '/Users/yasith/projects/artisan/cybershuttle-experi

## Option A - Wait for Completion

In [ ]:
plan.join()

## Option B - Terminate Execution

In [ ]:
plan.stop()

## Option C - Monitor Files During Execution

Displaying the status and files generated by each replica (task)

In [ ]:
for task in plan.tasks:
    status = task.status()
    files = task.files()
    
    print(status, files)

Displaying the intermediate results generated by each replica (task)

In [ ]:
from matplotlib import pyplot as plt
import pandas as pd

for index, task in enumerate(plan.tasks):

    @cs.task_context(task)
    def visualize():
        data = pd.read_csv("data.csv")
        plt.figure(figsize=(8, 6))
        plt.plot(data["x"], data["y"], marker="o", linestyle="-", linewidth=2, markersize=6)
        plt.title(f"Plot for Replica {index} of {len(plan.tasks)}")

    visualize()

## Questions to Resolve

In [ ]:
from cybershuttle.airavata import AiravataOperator

assert token is not None
op = AiravataOperator("/Users/yasith/projects/artisan/cybershuttle-experiment-sdk/settings.ini", token)

display(op.get_application_list())
display(op.get_compatible_deployments("NAMD_76b9b262-de09-4ab7-b4d4-e89502405af7"))

# display(op.upload_files(
#   host="gf4.ucs.indiana.edu",
#   port=9002,
#   username="pga",
#   local_input_path="./data",
#   base_path="",
#   project_name="Default Project",
#   experiment_name="NAMD_MD_SDK_20241127_0451",
#   auth_type="token"
# ))
display(op.upload_files(
  host="149.165.172.192",
  port=22,
  username="exouser",
  local_input_path="./data",
  base_path="/home/exouser",
  project_name="Default Project",
  experiment_name="NAMD_MD_SDK_20241127_0451",
  auth_type="pkey",
))